# import 

In [ ]:
%matplotlib inline

In [ ]:
import json

In [ ]:
import pandas 

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from pyiron import Project

In [ ]:
input_file = "input.json"
output_file = 'output.json'

In [ ]:
def pot_dict_to_pyiron_pot(pot_dict):
    return pandas.DataFrame({'Config': [pot_dict['config']],
                             'Filename': [pot_dict['filename']],
                             'Model': [pot_dict['model']],
                             'Name': [pot_dict['name']],
                             'Species': [pot_dict['species']]
                             })

In [ ]:
with open(input_file, 'r') as f:
    input_dict = json.load(f)

# project

In [ ]:
pr = Project(input_dict['path'])
potential = pot_dict_to_pyiron_pot(pot_dict=input_dict)
element = input_dict['element']

In [ ]:
pr.remove_jobs(recursive=True)

In [ ]:
structure = pr.create_ase_bulk(element, cubic=True)

# workflow

## minimize 

In [ ]:
job_mini = pr.create_job(pr.job_type.Lammps, 'lmp_mini')
job_mini.structure = structure
job_mini.potential = potential
job_mini.calc_minimize(pressure=0)
job_mini.server.run_mode.interactive = True 

## murngaghan

In [ ]:
job = pr.create_job(pr.job_type.Lammps, 'lmp_murn')
job.potential = potential
job.server.run_mode.interactive = True  

In [ ]:
murn = job.create_job(pr.job_type.Murnaghan, 'murn')

## phonons for each volume of the murnaghan

In [ ]:
job = pr.create_job(pr.job_type.Lammps, 'lmp_phono')
job.potential = potential
job.server.run_mode.interactive = True 

In [ ]:
phono = job.create_job(pr.job_type.PhonopyJob, 'phono')

In [ ]:
struct_master = phono.create_job(pr.job_type.StructureListMaster, 'struct_master')

## workflow configuration

In [ ]:
def step_one_to_two(job_1, job_2):
    job_2.structure = job_1.get_final_structure()

In [ ]:
def step_two_to_there(job_2, job_3):
    job_3.structure_lst = job_2.list_structures()

In [ ]:
job_lst_master = pr.create_job(pr.job_type.FlexibleMaster, 'lstmaster')
job_lst_master.append(job_mini)
job_lst_master.function_lst.append(step_one_to_two)
job_lst_master.append(murn)
job_lst_master.function_lst.append(step_two_to_there)
job_lst_master.append(struct_master)

# execution

In [ ]:
job_lst_master.run()

In [ ]:
df = pr.job_table()

# storage

In [ ]:
murn = pr.inspect('murn')

In [ ]:
output_dict = {'volume_lst': murn['output/volume'].tolist(), 
              'energy_lst': murn['output/energy'].tolist()}

In [ ]:
def store_equlibrium_output(murn, output_dict, key): 
    if key in murn['output'].list_nodes():
        output_dict[key] = murn['output/' + key]
    else: 
        output_dict[key] = 0.0

In [ ]:
store_equlibrium_output(murn=murn, output_dict=output_dict, key='equilibrium_b_prime')
store_equlibrium_output(murn=murn, output_dict=output_dict, key='equilibrium_bulk_modulus')
store_equlibrium_output(murn=murn, output_dict=output_dict, key='equilibrium_energy')
store_equlibrium_output(murn=murn, output_dict=output_dict, key='equilibrium_volume')

In [ ]:
for job_id in df[df.hamilton == 'PhonopyJob'].id.values:
    job = pr.inspect(job_id)
    output_dict[job.job_name] = [job['output/dos_energies'].tolist(), job['output/dos_total'].tolist()]

In [ ]:
with open(output_file, 'w') as f:
    json.dump(output_dict, f)